In [77]:
import pandas as pd
import yfinance as yf
from datetime import datetime

In [78]:
tickers = ['NVDA', 'SPY'] # As specify, Nnvdia stock and SPY ETF
num_expiries = 4 # Given next market closing day (Next Friday)
save_path = '../data/'

In [79]:
temp = 'NVDA'
yf_ticker = yf.Ticker(temp)
spot_price = yf_ticker.history(period="1d")['Close'].iloc[-1]
dividend_yield = yf_ticker.info.get('dividendYield', 0.0)

print(f"\n{temp} - Spot: {spot_price:.2f}, Dividend Yield: {dividend_yield}")

all_data = []

expiries = yf_ticker.options[:num_expiries]

for expiry in expiries:
    try:
        option_chain = yf_ticker.option_chain(expiry)
    except:
        print(f"Failed to fetch options for {temp} on {expiry}")
        continue

    for opt_type, df in [('call', option_chain.calls), ('put', option_chain.puts)]:
        df = df.copy()
        df['type'] = opt_type
        df['expiry'] = expiry
        df['spot'] = spot_price
        df['dividendYield'] = dividend_yield
        all_data.append(df)


NVDA - Spot: 109.67, Dividend Yield: 0.04


In [70]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)  # auto-expand width
pd.set_option('display.max_colwidth', None) # don't truncate long text

In [71]:
pd.reset_option('display.max_rows')
pd.reset_option('display.max_columns')

In [80]:
# Inspect Data added (sand box)
print(all_data[0].columns)
print(all_data[3][["contractSymbol","strike","expiry","type", "dividendYield"]])

Index(['contractSymbol', 'lastTradeDate', 'strike', 'lastPrice', 'bid', 'ask',
       'change', 'percentChange', 'volume', 'openInterest',
       'impliedVolatility', 'inTheMoney', 'contractSize', 'currency', 'type',
       'expiry', 'spot', 'dividendYield'],
      dtype='object')
         contractSymbol  strike      expiry type  dividendYield
0   NVDA250411P00050000    50.0  2025-04-11  put           0.04
1   NVDA250411P00055000    55.0  2025-04-11  put           0.04
2   NVDA250411P00060000    60.0  2025-04-11  put           0.04
3   NVDA250411P00065000    65.0  2025-04-11  put           0.04
4   NVDA250411P00070000    70.0  2025-04-11  put           0.04
..                  ...     ...         ...  ...            ...
56  NVDA250411P00155000   155.0  2025-04-11  put           0.04
57  NVDA250411P00160000   160.0  2025-04-11  put           0.04
58  NVDA250411P00165000   165.0  2025-04-11  put           0.04
59  NVDA250411P00170000   170.0  2025-04-11  put           0.04
60  NVDA250411

In [81]:
# Inspect full Calls and Puts df
sliced_data = [df.iloc[:, 2:] for df in all_data]
for i, df in enumerate(sliced_data):
    print(f"\n📄 Sliced Option Chain {i+1}")
    print(df)


📄 Sliced Option Chain 1
    strike  lastPrice    bid    ask    change  percentChange  volume  \
0     50.0      59.73  58.45  60.95 -2.810001      -4.493127     125   
1     55.0      54.70  53.50  55.95 -1.380001      -2.460772       1   
2     60.0      50.35  48.50  50.95 -3.380001      -6.290715       1   
3     65.0      45.10  43.50  45.95 -1.900001      -4.042556      15   
4     70.0      39.70  38.80  40.65 -2.700001      -6.367926      61   
..     ...        ...    ...    ...       ...            ...     ...   
75   220.0       0.01   0.00   0.01  0.000000       0.000000       1   
76   225.0       0.02   0.00   0.01  0.000000       0.000000     448   
77   230.0       0.01   0.00   0.01  0.000000       0.000000      84   
78   235.0       0.01   0.00   0.01  0.000000       0.000000       2   
79   240.0       0.01   0.00   0.01  0.000000       0.000000       6   

    openInterest  impliedVolatility  inTheMoney contractSize currency  type  \
0            213           2.34

In [76]:
for ticker in tickers:
    yf_ticker = yf.Ticker(ticker)
    spot_price = yf_ticker.history(period="1d")['Close'].iloc[-1]
    dividend_yield = yf_ticker.info.get('dividendYield', 0.0)

    print(f"\n{ticker} - Spot: {spot_price:.2f}, Dividend Yield: {dividend_yield}")

    all_data = []

    if ticker.upper() == 'SPY':
        expiries = ['2025-04-04', '2025-04-11', '2025-04-17', '2025-04-25']
    else:
        expiries = yf_ticker.options[:num_expiries]

    for expiry in expiries:
        try:
            option_chain = yf_ticker.option_chain(expiry)
        except:
            print(f"Failed to fetch options for {ticker} on {expiry}")
            continue

        for opt_type, df in [('call', option_chain.calls), ('put', option_chain.puts)]:
            df = df.copy() # ensures not editing the original DataFrame
            # Add sufficient Columns
            df['type'] = opt_type
            df['expiry'] = expiry # ensures if need to scale the data for increase reachness
            df['spot'] = spot_price # model Input as (S)
            df['dividendYield'] = dividend_yield # yield exist only for SPY, NVDA yield = 0.04
            all_data.append(df)
            
    options_df = pd.concat(all_data).reset_index(drop=True)
    filename = f"{save_path}options_{ticker.lower()}.csv"
    options_df.to_csv(filename, index=False)
    print(f"Saved: {filename}")


NVDA - Spot: 109.67, Dividend Yield: 0.04
Saved: ../data/options_nvda.csv

SPY - Spot: 555.66, Dividend Yield: 1.19
Saved: ../data/options_spy.csv


In [75]:
print("# of rows =" ,sum(len(df) for df in all_data))

# of rows = 1182
